# pandas.merge用法
用于按照某列列值来合并两个数据集

主要介绍`pandas.merge`的几个常见参数的用法
- `on`: 默认为`None`，选择按照两数据集的某个或几个**共有列**来合并数据集
- `suffixes`: 默认为`('_x', '_y')`，合并时，若数据集存在多个共有列，但on只选择了部分共有列来合并，其余共有列在合并时会被添加上后缀。
- `left_on`和`right_on`:默认为`None`，若两数据集用于合并的列列名不同，则分别指定。
- `how`: 默认为`'inner'`，按照合并列列值的并集合并。有(`'left'`, `'right'`, `'outer'`, `'inner'`) 四种取值。

In [1]:
import pandas as pd
help(pd.merge)
# merge(left, right, how='inner', on=None, 
#       left_on=None, right_on=None, left_index=False, 
#       right_index=False, sort=False, suffixes=('_x', '_y'), 
#       copy=True, indicator=False, validate=None)

Help on function merge in module pandas.core.reshape.merge:

merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
    Merge DataFrame or named Series objects with a database-style join.
    
    The join is done on columns or indexes. If joining columns on
    columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
    on indexes or indexes on a column or columns, the index will be passed on.
    
    Parameters
    ----------
    left : DataFrame
    right : DataFrame or named Series
        Object to merge with.
    how : {'left', 'right', 'outer', 'inner'}, default 'inner'
        Type of merge to be performed.
    
        * left: use only keys from left frame, similar to a SQL left outer join;
          preserve key order.
        * right: use only keys from right frame, similar to a SQL right outer join;
          preserve key 

先构建初始数据

In [2]:
left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                      'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                     'C': ['C0', 'C1', 'C2', 'C3'],
                     'D': ['D0', 'D1', 'D2', 'D3']})

In [3]:
left

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,B3


In [4]:
right

,key,C,D
0,K0,C0,D0
1,K1,C1,D1
2,K2,C2,D2
3,K3,C3,D3


## 参数on的用法
会默认以left和right都共有的列合并
- on的参数可以接收列表
- left和right共有列的值不同怎么办：只会合并left和right共有列的列值的交集
- 有多个共有列怎么办：没关系，一起合并，但left和right多列的值都要相等才合并

### 直接合并

In [5]:
# 直接合并
# 本例中同pd.merge(left, right, on='key')
pd.merge(left, right)

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


### 使用参数on合并

In [6]:
# 主动选择按照哪列合并，与上面相同
pd.merge(left, right, on='key')

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


### 演示：只合并on列列值的交集

In [7]:
# left和right只会合并on列列值的交集
right = pd.DataFrame({'key': ['K0', 'K1', 'K4', 'K5'],  # 构建非共有值
                     'C': ['C0', 'C1', 'C2', 'C3'],
                     'D': ['D0', 'D1', 'D2', 'D3']})
print('left key:',left.key.values)
print('right key:',right.key.values)
# 可以看到left和right的key的交集只有['K0' 'K1']

left key: ['K0' 'K1' 'K2' 'K3']
right key: ['K0' 'K1' 'K4' 'K5']


In [8]:
# 只会合并left和right的on列列值的交集
pd.merge(left, right, on='key')

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1


### 演示：若on列不是left和right的共有的

In [9]:
# 若on列不是不是left和right的共有的，会报错。
# pd.merge(left, right, on='A')
# KeyError: 'A'

### 演示：on的参数接收列表的情况

In [10]:
# left和right有多个共有列
left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                      'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                      'A': ['A0', 'A1', 'A2', 'A4'],  # A列最后一个元素的值不一样
                     'C': ['C0', 'C1', 'C2', 'C3'],
                     'D': ['D0', 'D1', 'D2', 'D3']})

In [11]:
print('left key:',left.key.values)
print('right key:',right.key.values)
print('left A:',left.A.values)
print('right A:',right.A.values)
# 可以看到left和right的A的交集只有['A0' 'A1' 'A2']
# 虽然left和right的key相等，但要按最小原则合并

left key: ['K0' 'K1' 'K2' 'K3']
right key: ['K0' 'K1' 'K2' 'K3']
left A: ['A0' 'A1' 'A2' 'A3']
right A: ['A0' 'A1' 'A2' 'A4']


In [12]:
# on接收列表的情况
pd.merge(left, right, on=['key', 'A'])
# A3没有合并

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2


### 演示：存在多个共有列，但on只选择了一列

In [13]:
# 存在多个共有列，但on只选择了一列
pd.merge(left, right, on='key')
# 会把其他共有列也合并，但是会加上不同的后缀

,key,A_x,B,A_y,C,D
0,K0,A0,B0,A0,C0,D0
1,K1,A1,B1,A1,C1,D1
2,K2,A2,B2,A2,C2,D2
3,K3,A3,B3,A4,C3,D3


## 参数suffixes的用法

In [14]:
# 可以用参数suffixes来调整后缀
pd.merge(left, right, on='key', suffixes=('_left', '_right'))

,key,A_left,B,A_right,C,D
0,K0,A0,B0,A0,C0,D0
1,K1,A1,B1,A1,C1,D1
2,K2,A2,B2,A2,C2,D2
3,K3,A3,B3,A4,C3,D3


## 参数left_on,right_on的用法
适用于left和right中用于合并的列，列名不同的情况

其余特点跟on差不多，但合并后会多出一列冗余列

In [15]:
import pandas as pd
left = pd.DataFrame({'key1': ['K0', 'K1', 'K2', 'K3'],  # 列名不同
                      'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key2': ['K0', 'K1', 'K2', 'K3'],  # 列名不同
                     'C': ['C0', 'C1', 'C2', 'C3'],
                     'D': ['D0', 'D1', 'D2', 'D3']})


In [16]:
pd.merge(left, right, left_on='key1',right_on='key2')

,key1,A,B,key2,C,D
0,K0,A0,B0,K0,C0,D0
1,K1,A1,B1,K1,C1,D1
2,K2,A2,B2,K2,C2,D2
3,K3,A3,B3,K3,C3,D3


In [17]:
# 可以看到多出来一列多余的数据
pd.merge(left, right, left_on='key1',right_on='key2').drop('key2',axis=1)

,key1,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


### 若选了完全没有交集的情况
选错了


In [18]:
pd.merge(left, right, left_on='key1',right_on='C')

,key1,A,B,key2,C,D


## 参数how的用法

之前说了，left和right合并的时候按照on选择的列列值的交集合并

其实是因为参数`how`默认情况是`how='inner'` 表示取on列列值的交集，还有其余3个情况

- `how='inner'`: 取交集
- `how='outer'`: 取并集，并用nan填充
- `how='left'`: 左侧取全部，右侧取部分
- `how='right'`: 右侧取全部，左侧取部分

In [19]:
import pandas as pd
left = pd.DataFrame({'key1': ['K0', 'K1', 'K2', 'K3'],
                      'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key2': ['K0', 'K1', 'K4', 'K5'],
                     'C': ['C0', 'C1', 'C4', 'C5'],
                     'D': ['D0', 'D1', 'D4', 'D5']})

In [20]:
# 取key的并集
pd.merge(left, right, left_on='key1',right_on='key2',how='inner')

,key1,A,B,key2,C,D
0,K0,A0,B0,K0,C0,D0
1,K1,A1,B1,K1,C1,D1


In [21]:
# 取key的交集
pd.merge(left, right, left_on='key1',right_on='key2',how='outer')

,key1,A,B,key2,C,D
0,K0,A0,B0,K0,C0,D0
1,K1,A1,B1,K1,C1,D1
2,K2,A2,B2,NaN,NaN,NaN
3,K3,A3,B3,NaN,NaN,NaN
4,NaN,NaN,NaN,K4,C4,D4
5,NaN,NaN,NaN,K5,C5,D5


In [22]:
# 只保证左边完整
pd.merge(left, right, left_on='key1',right_on='key2',how='left')

,key1,A,B,key2,C,D
0,K0,A0,B0,K0,C0,D0
1,K1,A1,B1,K1,C1,D1
2,K2,A2,B2,NaN,NaN,NaN
3,K3,A3,B3,NaN,NaN,NaN


In [23]:
# 只保证右边完整
pd.merge(left, right, left_on='key1',right_on='key2',how='right')

,key1,A,B,key2,C,D
0,K0,A0,B0,K0,C0,D0
1,K1,A1,B1,K1,C1,D1
2,NaN,NaN,NaN,K4,C4,D4
3,NaN,NaN,NaN,K5,C5,D5


## 次要参数indicator的使用

给合并后的数据添加一个参数显示数据是怎么合并的

In [24]:
pd.merge(left, right, left_on='key1',right_on='key2', 
         how = 'outer',
         indicator=True)

,key1,A,B,key2,C,D,_merge
0,K0,A0,B0,K0,C0,D0,both
1,K1,A1,B1,K1,C1,D1,both
2,K2,A2,B2,NaN,NaN,NaN,left_only
3,K3,A3,B3,NaN,NaN,NaN,left_only
4,NaN,NaN,NaN,K4,C4,D4,right_only
5,NaN,NaN,NaN,K5,C5,D5,right_only


## 其他例子

In [27]:
left = pd.DataFrame({'key': ['K0', 'K0', 'K1', 'K1', 'K1'],
                      'A': ['A0', 'A1', 'A2', 'A3', 'A4'],
                    'B': ['B0', 'B1', 'B2', 'B3', 'B4']})

right = pd.DataFrame({'key': ['K0', 'K1'],
                     'C': ['C0', 'C1'],
                     'D': ['D0', 'D1']})

In [28]:
pd.merge(left, right, on='key',how='left')

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K0,A1,B1,C0,D0
2,K1,A2,B2,C1,D1
3,K1,A3,B3,C1,D1
4,K1,A4,B4,C1,D1
